In [1]:
import timeit
import matplotlib.pyplot as plt
import pickle
from mnist import MNIST
from time import time

In [2]:
print("Loading dataset....")
mndata = MNIST('../mnist_dataset')
X_train, y_train = mndata.load_training()
X_train = (mndata.process_images_to_numpy(X_train)/255)
X_test, y_test = mndata.load_testing()
X_test = (mndata.process_images_to_numpy(X_test)/255)
print("Done.")

Loading dataset....
Done.


In [3]:
def one_hot_encode(labels):
    y_inp=np.zeros((len(labels),10))
    for ind,val in enumerate(labels):
        y_inp[ind][val]=1
    return y_inp

In [4]:
def batch_gen(st=0,size=20,validate=False):
    st=st%60000
    if validate:
        X=X_test[st:st+size].reshape(-1,28*28*1)
        labels=y_test[st:st+size]
    else:
        X=X_train[st:st+size].reshape(-1,28*28*1)
        labels=y_train[st:st+size]
    y=one_hot_encode(labels)
    return X,y

In [5]:
from nnet.network import Sequential,layers
from nnet.layers import conv2d,max_pool,flatten,dense,dropout,Activation
from nnet import functions
from nnet import optimizers
import numpy as np

Seed: 428
Seed: 396


In [6]:
model=Sequential()
model.add(dense(128,input_shape=(784),activation=functions.sigmoid))
model.add(dense(32,activation=functions.sigmoid))
model.add(dense(10))
model.add(Activation(functions.sigmoid))

In [7]:
model.summary()

⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽
Layer (type)               Output Shape             Activation        Param #
0 input_layer(InputLayer) (None, 784)                echo             0
__________________________________________________________________________________________
1 dense(dense)            (None, 128)                sigmoid          100480
__________________________________________________________________________________________
2 dense(dense)            (None, 32)                 sigmoid          4128
__________________________________________________________________________________________
3 dense(dense)            (None, 10)                 echo             330
__________________________________________________________________________________________
4 Activation(Activation)  (None, 10)                 sigmoid          0
Total Params: 104,938
Trainable Params: 104,938
Non-trainable Params: 0


In [8]:
model.compile(optimizer=optimizers.adam,loss=functions.mean_squared_error,learning_rate=0.001)

In [9]:
def validate():
    vdsz=2000
    dvme=10000//vdsz
    acc=0
    cross_entropy_loss=0
    for smpl in range(0,10000,vdsz):
        print("\rCalculating Validation acc...",smpl//vdsz+1,end='')
        inp,y_inp=batch_gen(smpl,size=vdsz,validate=True)
        logits=model.predict(inp)
        ans=logits.argmax(axis=1)
        cor=y_inp.argmax(axis=1)
        acc+=100*(ans==cor).mean()
        cross_entropy_loss+=model.loss(logits,labels=y_inp).mean()*10
    print("\rValidation Acc: {:.3f} %        Val loss: {:.8f}".format(acc/dvme,cross_entropy_loss/dvme))
    logits=model.predict(inp[:1])

In [10]:
def run():
    st=0
    btsz=128
    acc_tn=0
    _cc=0
    loss_tn=0
    while st<=60000:
        sam_tm=time()
        perc=st/600
        ck=np.random.randint(0,60000-btsz)
        inp,y_inp=batch_gen(ck,size=btsz)
        logits=model.fit(inp,y_inp)
        ans=logits.argmax(axis=1)
        cor=y_inp.argmax(axis=1)
        acc=100*(ans==cor).mean()
        cross_entropy_loss=model.loss(logits=logits,labels=y_inp).mean()*10
        acc_tn+=acc
        _cc+=1
        loss_tn+=cross_entropy_loss
        acc=acc_tn/_cc
        loss_=loss_tn/_cc
        if acc>=98:
            model.learning_rate=1e-5
        elif acc>=97:
            model.learning_rate=1e-4
#         elif acc>=95:
#             model.learning_rate=1e-4
        sam_tm=time()-sam_tm
        rem_sam=(60000-st)/btsz
        eta=int(rem_sam*sam_tm)
        print("\rProgress: {:.2f} %    Acc: {:.3f} %    loss: {:.6f}     Sample time: {:.3f}s    ETA: {}:{}s    .".format(perc,acc,loss_,sam_tm,eta//60,eta%60),end='')
        st+=btsz
    print("")

In [11]:
epochs=3
for epoch in range(epochs):
    print("EPOCH:",epoch+1,'/',epochs)
    st_tm=time()
    run()
    print("Epoch time: {}:{}s".format(int(time()-st_tm)//60,int(time()-st_tm)%60))
    validate()

EPOCH: 1 / 3
Progress: 99.84 %    Acc: 11.301 %    loss: 0.470415     Sample time: 0.015s    ETA: 0:0s    ..
Epoch time: 0:3s
Validation Acc: 11.350 %        Val loss: 0.44989031
EPOCH: 2 / 3
Progress: 99.84 %    Acc: 11.326 %    loss: 0.449862     Sample time: 0.018s    ETA: 0:0s    .
Epoch time: 0:3s
Validation Acc: 11.350 %        Val loss: 0.44987267
EPOCH: 3 / 3
Progress: 99.84 %    Acc: 11.172 %    loss: 0.449877     Sample time: 0.014s    ETA: 0:0s    .
Epoch time: 0:3s
Validation Acc: 11.350 %        Val loss: 0.44988079
